# Airline Arrivals

We have a dataset with details about flights. We need to predict whether the flight arrive is late or not. Flight could only be late if it's more than 30 minutes delayed. So we first need to check our data to find how we can check this.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv('C:/Users/vivek/Downloads/2005.csv')

In [3]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2005,1,28,5,1603.0,1605,1741.0,1759,UA,541,...,4,23,0,NaN,0,0,0,0,0,0
1,2005,1,29,6,1559.0,1605,1736.0,1759,UA,541,...,6,15,0,NaN,0,0,0,0,0,0
2,2005,1,30,7,1603.0,1610,1741.0,1805,UA,541,...,9,18,0,NaN,0,0,0,0,0,0
3,2005,1,31,1,1556.0,1605,1726.0,1759,UA,541,...,11,10,0,NaN,0,0,0,0,0,0
4,2005,1,2,7,1934.0,1900,2235.0,2232,UA,542,...,5,10,0,NaN,0,0,0,0,0,0


Our data contains a column 'ArrDelay' which shows how much delay did flight took to reach destination. We will simply check whether this column is more than 30 or not and store the result in new column for label.

In [4]:
df['Delayed'] = 0

In [5]:
df.at[df['ArrDelay'] < 30, 'Delayed'] = 0
df.at[df['ArrDelay'] > 30, 'Delayed'] = 1

In [6]:
df.columns

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
       'Delayed'],
      dtype='object')

Here we have alot of columns which we don't require so let's get rid of them.

In [7]:
df = df.drop(['ArrTime', 'TaxiIn', 'TaxiOut', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'Origin', 'Dest', 'ActualElapsedTime', 'TailNum', 'CancellationCode', 'ArrDelay', 'UniqueCarrier'],1)

In [8]:
df.describe()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,CRSArrTime,FlightNum,CRSElapsedTime,AirTime,DepDelay,Distance,Cancelled,Diverted,Delayed
count,7140596.0,7.140596e+06,7.140596e+06,7.140596e+06,7.006866e+06,7.140596e+06,7.140596e+06,7.140596e+06,7.140596e+06,6.992838e+06,7.006866e+06,7.140596e+06,7.140596e+06,7.140596e+06,7.140596e+06
mean,2005.0,6.481160e+00,1.571931e+01,3.944549e+00,1.344534e+03,1.337973e+03,1.499840e+03,2.042659e+03,1.259049e+02,1.012756e+02,8.674313e+00,7.237402e+02,1.872813e-02,1.964542e-03,1.150949e-01
std,0.0,3.410521e+00,8.785960e+00,1.989965e+00,4.767736e+02,4.642816e+02,4.804065e+02,1.841605e+03,6.975044e+01,8.246829e+01,3.119505e+01,5.711465e+02,1.355632e-01,4.427960e-02,3.191364e-01
min,2005.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,-4.000000e+01,-1.428000e+03,-1.199000e+03,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00
25%,2005.0,4.000000e+00,8.000000e+00,2.000000e+00,9.330000e+02,9.300000e+02,1.120000e+03,5.840000e+02,7.500000e+01,5.400000e+01,-4.000000e+00,3.160000e+02,0.000000e+00,0.000000e+00,0.000000e+00
50%,2005.0,6.000000e+00,1.600000e+01,4.000000e+00,1.331000e+03,1.328000e+03,1.523000e+03,1.446000e+03,1.070000e+02,8.300000e+01,0.000000e+00,5.620000e+02,0.000000e+00,0.000000e+00,0.000000e+00
75%,2005.0,9.000000e+00,2.300000e+01,6.000000e+00,1.735000e+03,1.725000e+03,1.912000e+03,3.172000e+03,1.550000e+02,1.300000e+02,7.000000e+00,9.500000e+02,0.000000e+00,0.000000e+00,0.000000e+00
max,2005.0,1.200000e+01,3.100000e+01,7.000000e+00,2.805000e+03,2.359000e+03,2.359000e+03,9.584000e+03,6.600000e+02,1.956000e+03,1.930000e+03,4.962000e+03,1.000000e+00,1.000000e+00,1.000000e+00


Dataset used here contains 7140596 rows so it's a huge dataset. For this reason we won't be comparing different machine learning algorithms because of less computational power.

In [9]:
df = df.dropna()

In [10]:
X = df.drop('Delayed', 1)
Y = df['Delayed']

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [12]:
rfc = RandomForestClassifier(n_estimators=10)
param_grid = {
    'n_estimators': [10, 20],
    'max_features': ['auto', 'sqrt']
}
CV_rfc = GridSearchCV(rfc, param_grid=param_grid, cv=3)
CV_rfc.fit(X_train, Y_train)
print("Best parameters: ", CV_rfc.best_params_)
print("Train data score: ", CV_rfc.score(X_train, Y_train))
print("Test data score: ", CV_rfc.score(X_test, Y_test))

Best parameters:  {'max_features': 'sqrt', 'n_estimators': 20}
Train data score:  0.9983301008193527
Test data score:  0.9661796923710538


Model is doing a pretty descent job here with 96% test score accuracy and 99% train score accuracy. May be size of our data is responsible for such successful model. Now let's try decreasing it's dimensionality using PCA and check how score is affected.

### PCA

In [13]:
pca = PCA(.95)
pca.fit(X_train)
x_pca = pca.transform(X_train)

In [15]:
rfc = RandomForestClassifier(n_estimators=20, max_features='sqrt')
rfc.fit(x_pca, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [16]:
x_test_pca = pca.transform(X_test)
print("Models score on train data: ", rfc.score(x_pca, Y_train))
print("Models score on test data: ", rfc.score(x_test_pca, Y_test))

Models score on train data:  0.9973039079225448
Models score on test data:  0.9334404905589145


So PCA clearly lowered model accuracy on test data by 3% which would not be acceptable in production environment. Though train dataset is still performing similar to previous model.